### Week 1 exercises

In [7]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [8]:
# Preamble
import sys
sys.path.append('..')

In [9]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("copenlu/answerable_tydiqa")

train_set = dataset["train"]
validation_set = dataset["validation"]

df_train = train_set.to_pandas()
df_val = validation_set.to_pandas()

df_train.head()


,question_text,document_title,language,annotations,document_plaintext,document_url
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...
2,వేప చెట్టు యొక్క శాస్త్రీయ నామం ఏమిటి?,వేప,telugu,"{'answer_start': [12], 'answer_text': ['Azadir...","వేప (లాటిన్ Azadirachta indica, syn. Melia aza...",https://te.wikipedia.org/wiki/%E0%B0%B5%E0%B1%...
3,চেঙ্গিস খান কোন বংশের রাজা ছিলেন ?,চেঙ্গিজ খান,bengali,"{'answer_start': [414], 'answer_text': ['বোরজি...",চেঙ্গিজ খান (মঙ্গোলীয়: Чингис Хаан আ-ধ্ব-ব: ...,https://bn.wikipedia.org/wiki/%E0%A6%9A%E0%A7%...
4,రెయ్యలగడ్ద గ్రామ విస్తీర్ణత ఎంత?,రెయ్యలగడ్ద,telugu,"{'answer_start': [259], 'answer_text': ['27 హె...","రెయ్యలగడ్ద, విశాఖపట్నం జిల్లా, గంగరాజు మాడుగుల...",https://te.wikipedia.org/wiki/%E0%B0%B0%E0%B1%...


In [10]:
# Get train and validation data for each language
df_train_bengali = df_train[df_train['language'] == 'bengali']
df_train_arabic = df_train[df_train['language'] == 'arabic']
df_train_indonesian = df_train[df_train['language'] == 'indonesian']

df_val_bengali = df_val[df_val['language'] == 'bengali']
df_val_arabic = df_val[df_val['language'] == 'arabic']
df_val_indonesian = df_val[df_val['language'] == 'indonesian']

print(len(df_train_bengali))
print(len(df_train_arabic))
print(len(df_train_indonesian))

# For testing
df_val_english = df_val[df_val['language'] == 'english']
df_train_english = df_train[df_train['language'] == 'english']


4779
29598
11394


In [11]:
df_train_bengali_document = df_train[df_train['language'] == 'bengali']["document_plaintext"]
df_train_arab_document = df_train[df_train['language'] == 'arabic']["document_plaintext"]
df_train_indonesian_document = df_train[df_train['language'] == 'indonesian']["document_plaintext"]
df_train_indonesian_document.head()

df_train_english_document = df_train[df_train['language'] == 'english']["document_plaintext"]


In [12]:
# Tokenize the documents
from transformers import AutoTokenizer
mbert_tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

def tokenize(df, key, transformer_model):
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]

# Tokinize train document_plaintext
tokenize(df_train_bengali, "document_plaintext", mbert_tokeniser)
tokenize(df_train_arabic, "document_plaintext", mbert_tokeniser)
tokenize(df_train_indonesian, "document_plaintext", mbert_tokeniser)

# Tokinize validation document_plaintext
tokenize(df_val_bengali, "document_plaintext", mbert_tokeniser)
tokenize(df_val_arabic, "document_plaintext", mbert_tokeniser)
tokenize(df_val_indonesian, "document_plaintext", mbert_tokeniser)


# For testing
tokenize(df_train_english, "document_plaintext", mbert_tokeniser)
tokenize(df_val_english, "document_plaintext", mbert_tokeniser)



(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)lingual-uncased/resolve/main/config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

(…)tilingual-uncased/resolve/main/vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

(…)gual-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-12-0760971b3fd8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]
<ipython-input-12-0760971b3fd8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]

In [13]:
# added in tokenization of the questions
# Tokinize train question_text
tokenize(df_train_bengali, "question_text", mbert_tokeniser)
tokenize(df_train_arabic, "question_text", mbert_tokeniser)
tokenize(df_train_indonesian, "question_text", mbert_tokeniser)

# Tokinize validation question_text
tokenize(df_val_bengali, "question_text", mbert_tokeniser)
tokenize(df_val_arabic, "question_text", mbert_tokeniser)
tokenize(df_val_indonesian, "question_text", mbert_tokeniser)

<ipython-input-12-0760971b3fd8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]
<ipython-input-12-0760971b3fd8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]
<ipython-input-12-0760971b3fd8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [14]:
# Training data document_plaintext tokenized
document_plaintext_tokenized_bengali = list(df_train_bengali["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_arabic = list(df_train_arabic["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_indonesian = list(df_train_indonesian["document_plaintext_tokenized"].explode())

# Validation data document_plaintext tokenized
document_plaintext_tokenized_val_bengali = list(df_val_bengali["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_val_arabic = list(df_val_arabic["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_val_indonesian = list(df_val_indonesian["document_plaintext_tokenized"].explode())

In [15]:
# Training data question_text tokenized
question_text_tokenized_bengali = list(df_train_bengali["question_text_tokenized"].explode())
question_text_tokenized_arabic = list(df_train_arabic["question_text_tokenized"].explode())
question_text_tokenized_indonesian = list(df_train_indonesian["question_text_tokenized"].explode())

# Validation data question_text tokenized
question_text_tokenized_val_bengali = list(df_val_bengali["question_text_tokenized"].explode())
question_text_tokenized_val_arabic = list(df_val_arabic["question_text_tokenized"].explode())
question_text_tokenized_val_indonesian = list(df_val_indonesian["question_text_tokenized"].explode())

In [33]:
total_ben = set(document_plaintext_tokenized_bengali + question_text_tokenized_bengali + document_plaintext_tokenized_val_bengali + question_text_tokenized_val_bengali)
print(f"vocab size bengali: {len(total_ben)}")
print(f"Fraction of unique tokens bengali: {len(total_ben)/len(document_plaintext_tokenized_bengali + question_text_tokenized_bengali + document_plaintext_tokenized_val_bengali + question_text_tokenized_val_bengali)}")

vocab size bengali: 6761
Fraction of unique tokens bengali: 0.005489985911661653


In [35]:
total_arab = set(document_plaintext_tokenized_arabic + question_text_tokenized_arabic + document_plaintext_tokenized_val_arabic + question_text_tokenized_val_arabic)
print(f"vocab size arabic: {len(total_arab)}")
print(f"Fraction of unique tokens arabic: {len(total_arab)/len(document_plaintext_tokenized_arabic + question_text_tokenized_arabic + document_plaintext_tokenized_val_arabic + question_text_tokenized_val_arabic)}")

vocab size bengali: 17403
Fraction of unique tokens bengali: 0.003066301029253812


In [36]:
total_indo = set(document_plaintext_tokenized_indonesian + question_text_tokenized_indonesian + document_plaintext_tokenized_val_indonesian + question_text_tokenized_val_indonesian)
print(f"vocab size indonesian: {len(total_indo)}")
print(f"Fraction of unique tokens indonesian: {len(total_indo)/len(document_plaintext_tokenized_indonesian + question_text_tokenized_indonesian + document_plaintext_tokenized_val_indonesian + question_text_tokenized_val_indonesian)}")

vocab size indonesian: 25660
Fraction of unique tokens indonesian: 0.015709582110422567


(b) For each of the languages Arabic, Bengali and Indonesian, report the 5 most common words in the documents from the training set. Then report the 5 most common words in the questions from the training set. What do you observe?

In [16]:
from collections import Counter

def top_5_words(word_list):
    word_count = Counter(word_list)
    top_5 = word_count.most_common(5)
    return top_5

In [17]:
print('Top 5 Bengali words:', top_5_words(question_text_tokenized_bengali))
print('Top 5 Arabic words:', top_5_words(question_text_tokenized_arabic))
print('Top 5 Indonesian words:', top_5_words(question_text_tokenized_indonesian))

Top 5 Bengali words: [('?', 4777), ('ক', 3552), ('##র', 1914), ('##া', 1625), ('স', 1558)]
Top 5 Arabic words: [('؟', 29576), ('ال', 18018), ('م', 10609), ('ما', 8139), ('##تى', 7138)]
Top 5 Indonesian words: [('?', 11368), ('apa', 3791), ('##kah', 2783), ('kap', 2339), ('##an', 2185)]


(c) implement an “oracle” function that indicates whether a question is an- swerable or not given the document and answer. That is, the function will output 1 if the answer to the question appears in the document and 0 otherwise. Then implement a rule-based classifier that predicts whether a question is answerable only using the document and question. Use the oracle function to evaluate it. What is the performance of your classifier on the validation set for each of the languages?

In [18]:
# Oracle function which takes a dataframe and row of a dataframe to check whether the text of the question appears in the document text
def oracle(df, row):
    """
    If text (a word) from question appears in document, assume that question is answerable
    Return 1 if answerable
    Return 0 if not answerable
    """

    question = df['question_text'][row].split()
    document = df['document_plaintext'][row].split()

    found = False
    for word in question:
        if word in document:
            found = True
            break

    if found:
        return 1
    else:
        return 0

In [19]:
# Creating a column with whether the oracle function classifies the result as 0 or 1
answer_classification = []

for index, row in df_train.iterrows():
    result = oracle(df_train, index)
    answer_classification.append(result)

df_train['answer_classification'] = answer_classification

In [20]:
# Creating a binary column where if the question is answered it is equal to 1, and if not answerable it is 0
def check_annotations(annotation):
    return annotation == {'answer_start': [-1], 'answer_text': ['']}

df_train['correct_answer'] = df_train['annotations'].apply(check_annotations)
df_train['correct_answer'] = (~df_train['correct_answer']).astype(int)

In [21]:
# Performance metrics function
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def performance_metrics(df):
    y_true = df['correct_answer']
    y_pred = df['answer_classification']

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    metrics_dict = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1}

    return metrics_dict

In [22]:
# Update other languages dataframes
df_train_bengali = df_train[df_train['language'] == 'bengali']
df_train_arabic = df_train[df_train['language'] == 'arabic']
df_train_indonesian = df_train[df_train['language'] == 'indonesian']

In [23]:
# display performance metrics
print('Overall:', performance_metrics(df_train))
print('Bengali:', performance_metrics(df_train_bengali))
print('Arabic:', performance_metrics(df_train_arabic))
print('Indonesian:', performance_metrics(df_train_indonesian))

Overall: {'Accuracy': 0.6483151972567569, 'Precision': 0.6254913378948901, 'Recall': 0.740023079970375, 'F1-score': 0.677954066698751}
Bengali: {'Accuracy': 0.686336053567692, 'Precision': 0.6260968015850552, 'Recall': 0.9255230125523013, 'F1-score': 0.746918791153132}
Arabic: {'Accuracy': 0.6703493479289141, 'Precision': 0.617122969837587, 'Recall': 0.8982776089159068, 'F1-score': 0.7316187594553707}
Indonesian: {'Accuracy': 0.6614885027207302, 'Precision': 0.6140437631351218, 'Recall': 0.8710978603998597, 'F1-score': 0.7203248495395548}
